## Loading Packages

In [1]:
import pandas as pd
import os
import sys
import gzip

In [2]:
### data path
path0 = 'Data/'
filename = 'reviews_Amazon_Instant_Video_5.json.gz'
path = path0 + filename

In [3]:
### function to extract and parse the data

def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield eval(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

df = getDF(path)

In [4]:
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A11N155CW1UV02,B000H00VBQ,AdrianaM,"[0, 0]",I had big expectations because I love English ...,2.0,A little bit boring for me,1399075200,"05 3, 2014"
1,A3BC8O2KCL29V2,B000H00VBQ,Carol T,"[0, 0]",I highly recommend this series. It is a must f...,5.0,Excellent Grown Up TV,1346630400,"09 3, 2012"
2,A60D5HQFOTSOM,B000H00VBQ,"Daniel Cooper ""dancoopermedia""","[0, 1]",This one is a real snoozer. Don't believe anyt...,1.0,Way too boring for me,1381881600,"10 16, 2013"
3,A1RJPIGRSNX4PW,B000H00VBQ,"J. Kaplan ""JJ""","[0, 0]",Mysteries are interesting. The tension betwee...,4.0,Robson Green is mesmerizing,1383091200,"10 30, 2013"
4,A16XRPF40679KG,B000H00VBQ,Michael Dobey,"[1, 1]","This show always is excellent, as far as briti...",5.0,Robson green and great writing,1234310400,"02 11, 2009"


### Data Wrangling

#### review Text

In [5]:
X = df['reviewText']

In [6]:
y = df['overall']

In [7]:
y1 = df['summary']

#### data cleaning

In [8]:
import re

In [9]:
X.iloc[18]

'Enjoyed some of the comedians, it was a joy to laugh after losing my father whom I was a caregiver for'

#### a) Cleaning Special Characters and Removing Punctuations:

In [19]:
from string import punctuation

In [20]:
punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [11]:
# Some preprocesssing that will be common to all the text classification methods you will see. 

puncts = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£', 
 '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', 
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', 
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', 
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', ]

def clean_text(x):
    x = str(x)
    for punct in puncts:
        if punct in x:
            x = x.replace(punct, f' {punct} ')
    return x

In [12]:
X = X.apply(lambda x: clean_text(x))

In [15]:
import torch

In [30]:
from torchtext.data import Field, Dataset, Example
import pandas as pd

In [138]:
class DataFrameDataset(Dataset):
    """Class for using pandas DataFrames as a datasource"""
    def __init__(self, examples, fields, filter_pred=None):
        """
        Create a dataset from a pandas dataframe of examples and Fields
        Arguments:
         examples pd.DataFrame: DataFrame of examples
         fields {str: Field}: The Fields to use in this tuple. The
             string is a field name, and the Field is the associated field.
         filter_pred (callable or None): use only exanples for which
             filter_pred(example) is true, or use all examples if None.
             Default is None
        """
        self.examples = examples.apply(SeriesExample.fromSeries, args=(fields,), axis=1).tolist()
        if filter_pred is not None:
            self.examples = filter(filter_pred, self.examples)
        self.fields = dict(fields)
        # Unpack field tuples
        for n, f in list(self.fields.items()):
             if isinstance(n, tuple):
                    self.fields.update(zip(n, f))
                    del self.fields[n]

class SeriesExample(Example):
    """Class to convert a pandas Series to an Example"""
    @classmethod
    def fromSeries(cls, data, fields):
        return cls.fromdict(data.to_dict(), fields)

    @classmethod
    def fromdict(cls, data, fields):
        ex = cls()

        for key, field in fields.items():
            if key not in data:
                raise ValueError("Specified key {} was not found in "
                         "the input data".format(key))
            if field is not None:
                setattr(ex, key, field.preprocess(data[key]))
            else:
                setattr(ex, key, data[key])
        return ex
    
    def __str__(self):
        return str(self.reviewText)

In [140]:
from torchtext.data import LabelField, Field
TEXT = Field(tokenize='spacy')
LABEL = LabelField(dtype=torch.float)
ds = DataFrameDataset(df, {'reviewText': TEXT, 'overall': LABEL})

In [141]:
train_data, test_data = ds.split(random_state = random.seed(SEED))

In [142]:
train_data, valid_data = train_data.split(random_state = random.seed(SEED))

In [143]:
MAX_VOCAB_SIZE = 25_000

TEXT.build_vocab(train_data, max_size = MAX_VOCAB_SIZE)
LABEL.build_vocab(train_data)

In [144]:
print(TEXT.vocab.freqs.most_common(20))

[('.', 91604), ('the', 80780), (',', 69030), ('and', 48842), ('a', 41598), ('to', 41079), ('of', 37531), ('I', 34246), ('is', 32192), (' ', 30126), ('it', 23554), ('in', 21008), ('that', 18869), ('this', 17716), ("'s", 14395), ('for', 13264), ('with', 13043), ('The', 12307), ('show', 12038), ('-', 11934)]


In [145]:
print(TEXT.vocab.itos[:10])

['<unk>', '<pad>', '.', 'the', ',', 'and', 'a', 'to', 'of', 'I']


In [146]:
print(LABEL.vocab.stoi)

defaultdict(None, {5.0: 0, 4.0: 1, 3.0: 2, 2.0: 3, 1.0: 4})


In [147]:
BATCH_SIZE = 64
from torchtext import data
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    device = device)

In [148]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        
        super().__init__()
        
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        
        self.rnn = nn.RNN(embedding_dim, hidden_dim)
        
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, text):

        #text = [sent len, batch size]
        
        embedded = self.embedding(text)
        
        #embedded = [sent len, batch size, emb dim]
        
        output, hidden = self.rnn(embedded)
        
        #output = [sent len, batch size, hid dim]
        #hidden = [1, batch size, hid dim]
        
        assert torch.equal(output[-1,:,:], hidden.squeeze(0))
        
        return self.fc(hidden.squeeze(0))

In [149]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1

model = RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)

In [150]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')


The model has 2,592,105 trainable parameters


In [151]:
import torch.optim as optim

optimizer = optim.SGD(model.parameters(), lr=1e-3)

In [152]:
criterion = nn.BCEWithLogitsLoss()

In [153]:
model = model.to(device)
criterion = criterion.to(device)


In [154]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [155]:

def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
                
        predictions = model(batch.reviewText).squeeze(1)
        
        loss = criterion(predictions, batch.overall)
        
        acc = binary_accuracy(predictions, batch.overall)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [156]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.reviewText).squeeze(1)
            
            loss = criterion(predictions, batch.overall)
            
            acc = binary_accuracy(predictions, batch.overall)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [157]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [158]:
N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

TypeError: '<' not supported between instances of 'SeriesExample' and 'SeriesExample'

#### checking the features

In [24]:
from sklearn.feature_extraction.text import CountVectorizer

In [26]:
from sklearn.model_selection import train_test_split

In [37]:
text_train, text_test, y_train, y_text = train_test_split(X, y, stratify=y, random_state =42, test_size=.2)

In [47]:
vectorizer = CountVectorizer(min_df=3)

In [48]:
vectorizer.fit(text_train)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=3,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [49]:
X_train = vectorizer.transform(text_train)
X_test = vectorizer.transform(text_test)

In [50]:
X_train.shape

(29700, 22091)

In [51]:
print(vectorizer.get_feature_names()[:1111])

['00', '000', '007', '01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '100', '1000', '100th', '101', '102', '104', '105', '108', '1080p', '10pm', '10th', '11', '110', '112', '116', '117', '11th', '12', '12th', '13', '13th', '14', '143', '14th', '15', '150', '15th', '16', '1600', '165', '169', '16mm', '16th', '17', '1776', '17th', '18', '180', '1800', '1800s', '1844', '1860', '1864', '1865', '1890', '18th', '19', '1900', '1900s', '1906', '1908', '1914', '1920', '1920s', '1930', '1930s', '1931', '1940', '1940s', '1945', '1950', '1950s', '1956', '1959', '1960', '1960s', '1962', '1963', '1964', '1965', '1966', '1967', '1968', '1969', '1970', '1970s', '1971', '1972', '1973', '1975', '1976', '1977', '1978', '1979', '1980', '1980s', '1981', '1982', '1983', '1984', '1985', '1986', '1987', '1988', '1989', '1990', '1990s', '1991', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '19th', '1hr', '1st', '20', '200', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '

In [52]:
print(vectorizer.get_feature_names()[-1111:])

['vamp', 'vampire', 'vampires', 'vampiric', 'vampirism', 'vamps', 'van', 'vancamp', 'vance', 'vancouver', 'vanderbilt', 'vandervoort', 'vanessa', 'vanilla', 'vanish', 'vanished', 'vanishes', 'vanishing', 'vanity', 'vantage', 'vapid', 'variable', 'variant', 'variation', 'variations', 'varied', 'varies', 'varieties', 'variety', 'various', 'varma', 'vary', 'varying', 'vast', 'vastly', 'vaughn', 'vault', 'vcr', 've', 'vector', 'vectors', 'veep', 'veer', 'veered', 'veering', 'veers', 'vega', 'vegan', 'veganism', 'vegans', 'vegas', 'vegetable', 'vegetables', 'vegetarian', 'vehicle', 'vehicles', 'veil', 'veiled', 'vein', 'veins', 'velez', 'velvet', 'venal', 'vendetta', 'veneer', 'venerable', 'vengeance', 'vengeful', 'venice', 'venom', 'ventimiglia', 'ventriloquist', 'venture', 'ventured', 'ventures', 'venturing', 'venue', 'venues', 'venus', 'vera', 'verbal', 'verbally', 'verbatim', 'verdict', 'verdictread', 'vergara', 'verge', 'verges', 'verging', 'veridian', 'verified', 'verify', 'verisimili

In [53]:
def clean_numbers(x):
    if bool(re.search(r'\d', x)):
        x = re.sub('[0-9]{5,}', '#####', x)
        x = re.sub('[0-9]{4}', '####', x)
        x = re.sub('[0-9]{3}', '###', x)
        x = re.sub('[0-9]{2}', '##', x)
    return x

c) Removing Misspells:

In [56]:
# This comes from CPMP script in the Quora questions similarity challenge. 
import re
from collections import Counter
import gensim
import heapq
from operator import itemgetter
from multiprocessing import Pool

model = gensim.models.KeyedVectors.load_word2vec_format('../input/embeddings/GoogleNews-vectors-negative300/GoogleNews-vectors-negative300.bin', 
                                                        binary=True)
words = model.index2word

w_rank = {}
for i,word in enumerate(words):
    w_rank[word] = i

WORDS = w_rank

def words(text): return re.findall(r'\w+', text.lower())

def P(word): 
    "Probability of `word`."
    # use inverse of rank as proxy
    # returns 0 if the word isn't in the dictionary
    return - WORDS.get(word, 0)

def correction(word): 
    "Most probable spelling correction for word."
    return max(candidates(word), key=P)

def candidates(word): 
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

def build_vocab(texts):
    sentences = texts.apply(lambda x: x.split()).values
    vocab = {}
    for sentence in sentences:
        for word in sentence:
            try:
                vocab[word] += 1
            except KeyError:
                vocab[word] = 1
    return vocab

vocab = build_vocab(train.question_text)

top_90k_words = dict(heapq.nlargest(90000, vocab.items(), key=itemgetter(1)))

pool = Pool(4)
corrected_words = pool.map(correction,list(top_90k_words.keys()))

for word,corrected_word in zip(top_90k_words,corrected_words):
    if word!=corrected_word:
        print(word,":",corrected_word)

ModuleNotFoundError: No module named 'gensim'

In [55]:
! pip install gensim

     |████████████████████████████████| 24.7MB 7.7MB/s eta 0:00:01     |█████████████████▏              | 13.3MB 3.1MB/s eta 0:00:04     |█████████████████████           | 16.2MB 3.1MB/s eta 0:00:03
     |████████████████████████████████| 71kB 7.9MB/s eta 0:00:01
     |████████████████████████████████| 133kB 8.1MB/s eta 0:00:01
     |████████████████████████████████| 71kB 7.0MB/s eta 0:00:011
     |████████████████████████████████| 5.7MB 9.7MB/s eta 0:00:01
  Stored in directory: /Users/ZYe/Library/Caches/pip/wheels/5f/ea/fb/5b1a947b369724063b2617011f1540c44eb00e28c3d2ca8692
Successfully built smart-open
You should consider upgrading via the 'pip install --upgrade pip' command.
